### Preprocess

In [61]:
import pandas as pd

df = pd.read_csv('/workspace/competitions/Sly/Duy_NCKH_2025/sample/168.csv')

In [13]:
# Tạo series nối chunk theo article_position
full_article_map = df.groupby('article_position')['Chunk'].apply(lambda chunks: '\n'.join(chunks))

# Map giá trị nối này vào từng dòng dựa trên article_position
df['full_article_content'] = df['article_position'].map(full_article_map)


In [15]:
df.to_csv('output_with_full_article_content.csv', index=False, encoding='utf-8-sig')


In [62]:
df = pd.read_csv('/workspace/competitions/Sly/Duy_NCKH_2025/source/my_utils/output_with_full_article_content.csv')

### Prompt

In [103]:
CONTEXTUAL_PROMPT = """\
Dưới đây là thông tin bổ sung về ngữ cảnh của chunk này:

- **Tên nghị định**: {DECREE_NAME}
- **Tên chương**: {CHAPTER_NAME}
- **Vị trí của khoản trong điều**: {SARTICLE_POSITION}
- **Toàn bộ nội dung của điều chứa chunk**: {ARTICLE_TITLE}:\n{FULL_ARTICLE_CONTENT}

---

### Chunk đầu vào:
{CHUNK_CONTENT}

---

### Yêu cầu:
Hãy viết **một câu ngắn gọn** nhằm mô tả ngữ cảnh tổng quát của chunk, giúp định vị nội dung trong toàn bộ tài liệu nhằm cải thiện khả năng tìm kiếm.

**Yêu cầu cụ thể:**
- Không lặp lại nguyên văn nội dung chunk.
- Không cần nêu rõ số điều, khoản, nghị định.
- Không liệt kê các chi tiết cụ thể hay hành vi rời rạc.
- Phải làm rõ **đối tượng áp dụng** của quy định (ví dụ xe thì là xe gì, người thì đó là cá nhân, cơ quan hay tổ chức nào).
- Câu mô tả cần mang tính tổng quát, đủ rõ ràng để người đọc hình dung được chủ đề chính.

**Chỉ trả lời đúng 1 câu mô tả nội dung chính của chunk, và trả lời dưới dạng JSON với cấu trúc sau:**

```json
{{"context": "Nội dung câu mô tả chính của chunk"}}
"""

### Testing:

In [104]:
from openai import OpenAI
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent))

from dotenv import load_dotenv
load_dotenv(override=True)

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [105]:
df.head()

,Chunk,chapter_title,chapter_position,article_title,article_position,sarticle_position,full_article_content
0,1. Nghị định này quy định về:\n\na) Xử phạt vi...,Chương I\r\n\r\nNHỮNG QUY ĐỊNH CHUNG,1.0,Điều 1. Phạm vi điều chỉnh\r\n\r,1,NaN,1. Nghị định này quy định về:\n\na) Xử phạt vi...
1,"1. Cá nhân, tổ chức Việt Nam; cá nhân, tổ chức...",Chương I\r\n\r\nNHỮNG QUY ĐỊNH CHUNG,1.0,Điều 2. Đối tượng áp dụng\r\n\r,2,NaN,"1. Cá nhân, tổ chức Việt Nam; cá nhân, tổ chức..."
2,1. Đối với mỗi hành vi vi phạm hành chính về t...,Chương I\r\n\r\nNHỮNG QUY ĐỊNH CHUNG,1.0,"Điều 3. Hình thức xử phạt vi phạm hành chính, ...",3,1.0,1. Đối với mỗi hành vi vi phạm hành chính về t...
3,"2. Căn cứ vào tính chất, mức độ vi phạm, cá nh...",Chương I\r\n\r\nNHỮNG QUY ĐỊNH CHUNG,1.0,"Điều 3. Hình thức xử phạt vi phạm hành chính, ...",3,2.0,1. Đối với mỗi hành vi vi phạm hành chính về t...
4,3. Các biện pháp khắc phục hậu quả vi phạm hàn...,Chương I\r\n\r\nNHỮNG QUY ĐỊNH CHUNG,1.0,"Điều 3. Hình thức xử phạt vi phạm hành chính, ...",3,3.0,1. Đối với mỗi hành vi vi phạm hành chính về t...


In [106]:
def get_context(
    chunk: str,
    chapter_title: str,
    article_title: str,
    sarticle_position,
    full_article_content,
    decree_name: str = "Quy định xử phạt vi phạm hành chính về trật tự, an toàn giao thông trong lĩnh vực giao thông đường bộ; trừ điểm, phục hồi giấy phép lái xe",
):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.1,
        response_format={ 
            "type": "json_object"
        },
        messages = [
            {
                "role": "system",
                "content": (
                    "Bạn là một trợ lý pháp lý thông minh, có nhiệm vụ tạo context cho từng đoạn văn bản pháp luật (chunk) "
                    "để hỗ trợ truy vấn ngữ nghĩa (contextual RAG)."
                ),
            },
            {
                "role": "user",
                "content": CONTEXTUAL_PROMPT.format(
                    DECREE_NAME=decree_name,
                    CHAPTER_NAME=chapter_title,
                    ARTICLE_TITLE=article_title, 
                    SARTICLE_POSITION=sarticle_position,
                    FULL_ARTICLE_CONTENT=full_article_content,
                    CHUNK_CONTENT=chunk,
                ),
            },
        ]
    )
    return response.choices[0].message.content


In [107]:
for _, row in df[40:45].iterrows():
    chunk = row['Chunk']
    chapter_title = row['chapter_title']
    article_title = row['article_title'] 
    sarticle_position = row['sarticle_position']
    
    if pd.isna(sarticle_position):
        full_article_content = None
        print('na')
    else:
        full_article_content = row['full_article_content']
    
    contextual_content = get_context(chunk, chapter_title, article_title, sarticle_position, full_article_content)
    print(f"chunk: {chunk}\nfull_article_content: {article_title}:\n{full_article_content}\n\ncontext: {contextual_content}")
    print("\n\n----------------------------\n\n")

chunk: 11. Tịch thu phương tiện đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:

a) Buông cả hai tay khi đang điều khiển xe; dùng chân điều khiển xe; ngồi về một bên điều khiển xe; nằm trên yên xe điều khiển xe; thay người điều khiển khi xe đang chạy; quay người về phía sau để điều khiển xe hoặc bịt mắt điều khiển xe;

b) Điều khiển xe chạy bằng một bánh đối với xe hai bánh, chạy bằng hai bánh đối với xe ba bánh;

c) Tái phạm hành vi điều khiển xe lạng lách, đánh võng quy định tại điểm a khoản 9 Điều này.
full_article_content: Điều 7. Xử phạt, trừ điểm giấy phép lái của người điều khiển xe mô tô, xe gắn máy, các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy vi phạm quy tắc giao thông đường bộ:
1. Phạt tiền từ 200.000 đồng đến 400.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:

a) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại điểm b, điểm d, điểm